In [28]:
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")


In [56]:
import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime

crtfc_key='d04c4e316495dc1fe0628dd8a022490682676c56'

#회사정보 가져오기
#임의로 'E:/' 로 설정 및 다운로드 파일 corpcode.zip으로 설정
path='/Users/minjunglee/대학원/2023-가을학기/'
filename='corpcode.zip'

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
params = {
    'crtfc_key': crtfc_key,
}
results=requests.get(url, params=params)

file = open(path+filename, 'wb')
file.write(results.content)
file.close()

zipfile.ZipFile(path+filename).extractall(path)

tree = parse(path + 'CORPCODE.xml')
root = tree.getroot()
li=root.findall('list')
corp_code,corp_name,stock_code,modify_date=[],[],[],[]
for d in li:
    corp_code.append(d.find('corp_code').text)
    corp_name.append(d.find('corp_name').text)
    stock_code.append(d.find('stock_code').text)
    modify_date.append(d.find('modify_date').text)
corps_df = pd.DataFrame({'corp_code':corp_code,'corp_name':corp_name,
         'stock_code':stock_code,'modify_date':modify_date})

corps_df = corps_df.loc[corps_df['stock_code']!=' ',:].reset_index(drop=True)

In [59]:
#전체결과저장
result_all=[]
corf_detail=pd.DataFrame()

print('총 회사수는 : ' + str(corps_df.shape[0]))

# 수집한 회사에 대해서 for문.
for i, r in corps_df.iterrows():
    #if i == 2:
    #    break
    
    #없으면 어느 시점에서 에러발생
    time.sleep(0.2)
    
    # print('i = ' + str(i))
    corp_code=str(r['corp_code'])
    corp_name=r['corp_name']

    url = 'https://opendart.fss.or.kr/api/company.json'
    params = {
        'crtfc_key': crtfc_key,
        'corp_code' : corp_code,
    }

    results = requests.get(url, params=params, verify=False).json()
    # 응답이 정상 '000' 일 경우에만 데이터 수집
    if results['status'] == '000':
        result_all.append(results)    

corp_detail=pd.DataFrame(result_all)

총 회사수는 : 3646


In [60]:
corp_detail.to_csv('corps_detail_raw_data.csv')

In [61]:
corp_detail

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
1,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
2,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
3,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
4,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,000,정상,00624518,(주)다산솔루에타,"Dasan Solueta Co.,Ltd.",다산솔루에타,154040,양삼주,K,1351110065750,1238189952,경기도 화성시 마도면 해운로 498-42 -,www.solueta.com,http://www.solueta.com/kr/m51.php,031-508-2655,031-508-2606,2629,20030627,12
3642,000,정상,00624378,(주)마이크로투나노,"MICRO2NANO, INC.",마이크로투나노,424980,황규호,K,1101112059031,1198141599,경기도 용인시 처인구 이동읍 덕성산단2로50번길 43,micro2nano.com,,031-659-7900,031-659-7979,26129,20000828,12
3643,000,정상,00148610,한화투자증권(주),"HANWHA INVESTMENT&SECURITIES CO.,LTD",한화투자증권,003530,한두희,Y,1101110003155,1168119031,서울특별시 영등포구 여의대로 56 한화증권빌딩 한화투자증권 주식회사,www.koreastock.co.kr,,02-3772-7000,02-3772-7099,66121,19620719,12
3644,000,정상,00126414,삼성제약(주),"SAMSUNG PHARMACEUTICAL.CO.,Ltd.",삼성제약,001360,"김상재, 정성택",Y,1101110032568,2078100726,경기도 화성시 향남읍 제약공단4길 10 -,www.sspharm.co.kr,,031-353-6681,031-8016-2179,21210,19290815,12


In [62]:
import pandas as pd
from time import sleep

# 회사별 전체재무제표 확인
result_all = pd.DataFrame()


# bsns_year=['2023','2022','2021']
bsns_year=['2023']
# reprt_code=['11011','11014','11012','11013']#1분기보고서: 11013 반기보고서: 11012 3분기보고서: 11014 사업보고서: 11011
reprt_code=['11011']#1분기보고서: 11013 반기보고서: 11012 3분기보고서: 11014 사업보고서: 11011
fs_div='OFS'  #CFS:연결재무제표, OFS:재무제표

# corp_list = pd.DataFrame({'corp_code':['00126380'],'corp_name':['삼성전자'],'stock_code':['095570']})

for i, r in corp_detail.iterrows():
    
    #없으면 어느 시점에서 에러발생
    time.sleep(0.2)
    
    #전체재무제표 요청인자
    crtfc_key=crtfc_key
    corp_code=str(r['corp_code']).zfill(8)
    
    url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
    for x in bsns_year:
        for y in reprt_code:
            params = {
                'crtfc_key': crtfc_key,
                'corp_code' : corp_code,
                'bsns_year' : x,
                'reprt_code' : y,
                'fs_div' : fs_div,
            }

            #결과를 json형태로 저장
            results = requests.get(url, params=params,verify=False).json()

            if results['status'] == '000':
                result_df = pd.DataFrame(results['list'])
                result_all = pd.concat([result_all,result_df])
    
result_all

KeyboardInterrupt: 

In [63]:
result_all

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 45 기,114986815860,제 44 기,87669584025,제 43 기,80452873185,1,KRW,NaN
1,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 45 기,6795734003,제 44 기,11696458267,제 43 기,12375818651,2,KRW,NaN
2,20230915000256,11011,2023,00138747,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 45 기,399999985,제 44 기,3240120000,제 43 기,,3,KRW,NaN
3,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,-,제 45 기,46346281516,제 44 기,41674393205,제 43 기,39875166773,4,KRW,NaN
4,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_Inventories,재고자산,-,제 45 기,59508676713,제 44 기,29808239467,제 43 기,26447090836,5,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_DividendsPaid,배당금지급,별도재무제표 [member],제 6 기,-2713500000,제 5 기,-3015000000,제 4 기,-243210000,4,KRW,NaN
61,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본잉여금 [member],제 6 기,74207866689,제 5 기,74207866689,제 4 기,74218485078,5,KRW,NaN
62,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 6 기,-4432669487,제 5 기,4432669487,제 4 기,2061573423,5,KRW,NaN
63,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,별도재무제표 [member],제 6 기,88584101168,제 5 기,89875197202,제 4 기,92256911655,5,KRW,NaN


In [65]:
result_all.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',
       'account_id', 'account_nm', 'account_detail', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm',
       'bfefrmtrm_amount', 'ord', 'currency', 'thstrm_add_amount'],
      dtype='object')

In [84]:
result_df = result_all.pivot_table(index=['corp_code','bsns_year', 'reprt_code'], columns=['account_id'], values=['sj_div', 'sj_nm','account_nm', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm',
       'bfefrmtrm_amount', 'ord', 'currency'], aggfunc='first').stack(level=1).reset_index()


In [85]:
result_df

,corp_code,bsns_year,reprt_code,account_id,account_nm,bfefrmtrm_amount,bfefrmtrm_nm,currency,frmtrm_amount,frmtrm_nm,ord,sj_div,sj_nm,thstrm_amount,thstrm_nm
0,00106614,2023,11011,-표준계정코드 미사용-,순확정급여자산,1535815784,제 33 기,KRW,3168592243,제 34 기,20,BS,재무상태표,2823582300,제 35 기
1,00106614,2023,11011,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업으로부터 창출된 현금,16330239804,제 33 기,KRW,4911663583,제 34 기,1,CF,현금흐름표,-1927079110,제 35 기
2,00106614,2023,11011,dart_CapitalSurplus,기타불입자본,21791004500,제 33 기,KRW,21791004500,제 34 기,36,BS,재무상태표,21791004500,제 35 기
3,00106614,2023,11011,dart_CashAndCashEquivalentsAtBeginningOfPeriodCf,기초현금및현금성자산,24492678348,제 33 기,KRW,31933363331,제 34 기,33,CF,현금흐름표,10747455990,제 35 기
4,00106614,2023,11011,dart_CashAndCashEquivalentsAtEndOfPeriodCf,기말현금및현금성자산,31933363331,제 33 기,KRW,10747455990,제 34 기,34,CF,현금흐름표,11877971786,제 35 기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,01510489,2023,11011,ifrs-full_PurchaseOfIntangibleAssetsClassified...,무형자산의 취득,14732417,제 6 기,USD,9428932,제 7 기,9,CF,현금흐름표,17598651,제 8 기
2199,01510489,2023,11011,ifrs-full_PurchaseOfPropertyPlantAndEquipmentC...,유형자산의 취득,587927,제 6 기,USD,2162946,제 7 기,8,CF,현금흐름표,200115,제 8 기
2200,01510489,2023,11011,ifrs-full_RepaymentsOfBorrowingsClassifiedAsFi...,차입금의 상환,429007,제 6 기,USD,0,제 7 기,17,CF,현금흐름표,0,제 8 기
2201,01510489,2023,11011,ifrs-full_RetainedEarnings,결손금,-23553698,제 6 기,USD,-167758369,제 7 기,29,BS,재무상태표,-146238930,제 8 기


In [68]:
result_all['account_nm'].unique()

array(['유동자산', '현금및현금성자산', '단기금융상품', '매출채권 및 기타유동채권', '재고자산', '당기법인세자산',
       '기타유동금융자산', '기타유동자산', '비유동자산', '장기금융상품', '장기매출채권 및 기타비유동채권',
       '기타포괄손익-공정가치측정금융자산', '기타비유동금융자산', '유형자산', '영업권 이외의 무형자산', '사용권자산',
       '기타비유동자산', '자산총계', '유동부채', '매입채무 및 기타유동채무', '단기차입금', '당기법인세부채',
       '유동리스부채', '기타유동부채', '비유동부채', '퇴직급여부채', '비유동충당부채', '비유동리스부채',
       '이연법인세부채', '임대보증금', '부채총계', '자본금', '자본잉여금', '기타자본구성요소',
       '이익잉여금(결손금)', '자본총계', '자본과부채총계', '수익(매출액)', '매출원가', '매출총이익',
       '판매비와관리비', '영업이익(손실)', '기타이익', '기타손실', '금융수익', '금융원가',
       '법인세비용차감전순이익(손실)', '법인세비용', '당기순이익(손실)', '기타포괄손익',
       '당기손익으로 재분류되지 않는항목(세후기타포괄손익)', '확정급여제도의 재측정손익', '토지재평가잉여금',
       '토지재평가잉여금 법인세효과', '총포괄손익', '기본주당이익(손실)', '희석주당이익(손실)', '영업활동현금흐름',
       '당기순이익조정을 위한 가감', '외화환산손실', '감가상각비', '무형자산상각비', '퇴직급여', '대손상각비',
       '유형자산처분손실', '기타의 대손상각비', '이자비용', '외화환산이익', '유형자산처분이익',
       '당기손익-공정가치측정금융자산처분이익', '기타의 대손충당금환입', '이자수익', '영업활동으로인한자산ㆍ부채의변동',
       '매출채권의 감소(증가)', '미수금의 감소(증가)', '장기미수금의 감

In [86]:
# 원본 데이터 저장
import pickle

with open('fnlttAcntAll _raw_data.pickle','wb') as fw:
    pickle.dump(result_all,fw)

In [87]:
# 파일 저장
result_df.to_csv('fnlttAcntAll_raw_data.csv', index=False)